In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adamax
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
def read(txt, X, Y):
    fp = open(txt, 'r')
    for line in iter(fp):
        X_tmp,Y_tmp = [i for i in line.split()]
        X.append(X_tmp)
        Y.append(Y_tmp)
    fp.close
    return

def sift(des_list, X, Y):
    a = 0
    sift = cv2.xfeatures2d.SIFT_create()
    print("Calculating all the descriptors and keypoints...")
    for i in X:
        a += 1
        print("Calculating for image "+str(a)+"/"+str(len(X)), end="\r")
        
        im  = cv2.imread(i)
        im = cv2.resize(im, (64, 64))
        gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) 
        kp, des = sift.detectAndCompute(gray,None)
        des_list.append((i, kp, des))
    a2=0
    
    for j, j1, j2 in des_list[1:]:
        if j2 is None:
            a2 = a2+1
            del des_list[a2]
            del X[a2]
            del Y[a2]
            a2 = a2-1
        else:
            a2 = a2+1
            print("Calculating for image "+str(a2)+"/"+str(len(des_list)), end="\r")
    return

def onehot(Y_val):
    Y = to_categorical(Y_val)
    return Y

def knn_sift(des_list):
    a1 = 0
    print("Stacking all the descriptors in a numpy array...")
    descriptors = des_list[0][2]
    for image_path, keypoints, descriptor in des_list[1:]:
        a1=a1+1
        print("Calculating for image ["+ str(a1)+"/"+str(len(des_list)) +"]", end="\r")
        descriptors = np.vstack((descriptors, descriptor))
        
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    print("Done doing K-means")
    
    im_features = np.zeros((len(des_list), k), "float32")
    for i in range(len(des_list)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][2],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(des_list) + 1) / (1.0*nbr_occurences + 1)), 'float32')
    scaled_im_features = im_features * idf
            
    stdSlr = StandardScaler().fit(scaled_im_features)
    scaled_im_features = stdSlr.transform(scaled_im_features)
    
    return scaled_im_features

def svc_save(clf):
    joblib.dump((clf),"svc.pkl")
    return

def train_test(x, y):
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state = 42, test_size =0.2)
    return  X_train, X_test, Y_train, Y_test

model = Sequential()
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer = Adamax(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), metrics = ["accuracy"])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
X_train_sift = []
Y_train_sift = []
X_test_sift = []
Y_test_sift = []
X_val_sift = []
Y_val_sift = []
des_train_sift = []
des_test_sift = []        
des_val_sift = []
read('train.txt', X_train_sift, Y_train_sift)
read('test.txt', X_test_sift, Y_test_sift)
read('val.txt', X_val_sift, Y_val_sift)
sift(des_train_sift, X_train_sift, Y_train_sift)
sift(des_test_sift, X_test_sift, Y_test_sift)
sift(des_val_sift, X_val_sift, Y_val_sift)
Y_train_sift = onehot(Y_train_sift)
scaled_train_sift = knn_sift(des_train_sift)
scaled_test_sift = knn_sift(des_test_sift)
scaled_val_sift = knn_sift(des_val_sift)
X_train, X_test, Y_train, Y_test = train_test(scaled_train_sift, Y_train_sift)

Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Calculating all the descriptors and keypoints...
Stacking all the descriptors in a numpy array...
Done doing K-meansage [63316/63317]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [448/449]
Stacking all the descriptors in a numpy array...
Done doing K-meansage [449/450]


In [4]:
model.fit(X_train, Y_train, epochs= 100, validation_data=(X_test, Y_test), batch_size = 1)


Train on 50653 samples, validate on 12664 samples
Epoch 1/100
50653/50653 [==============================] - 73s 1ms/step - loss: 4.1794 - accuracy: 0.0237 - val_loss: 4.0196 - val_accuracy: 0.0302
Epoch 2/100
50653/50653 [==============================] - 72s 1ms/step - loss: 4.0173 - accuracy: 0.0319 - val_loss: 3.9313 - val_accuracy: 0.0392
Epoch 3/100
50653/50653 [==============================] - 72s 1ms/step - loss: 3.9286 - accuracy: 0.0405 - val_loss: 3.8736 - val_accuracy: 0.0464
Epoch 4/100
50653/50653 [==============================] - 72s 1ms/step - loss: 3.8666 - accuracy: 0.0467 - val_loss: 3.8308 - val_accuracy: 0.0501
Epoch 5/100
50653/50653 [==============================] - 72s 1ms/step - loss: 3.8173 - accuracy: 0.0536 - val_loss: 3.7979 - val_accuracy: 0.0540
Epoch 6/100
50653/50653 [==============================] - 72s 1ms/step - loss: 3.7776 - accuracy: 0.0597 - val_loss: 3.7730 - val_accuracy: 0.0561
Epoch 7/100
50653/50653 [==============================] - 72

50653/50653 [==============================] - 74s 1ms/step - loss: 3.5115 - accuracy: 0.1057 - val_loss: 3.6548 - val_accuracy: 0.0781
Epoch 55/100
50653/50653 [==============================] - 74s 1ms/step - loss: 3.5081 - accuracy: 0.1076 - val_loss: 3.6549 - val_accuracy: 0.0783
Epoch 56/100
50653/50653 [==============================] - 73s 1ms/step - loss: 3.5068 - accuracy: 0.1081 - val_loss: 3.6553 - val_accuracy: 0.0787
Epoch 57/100
50653/50653 [==============================] - 74s 1ms/step - loss: 3.5022 - accuracy: 0.1089 - val_loss: 3.6560 - val_accuracy: 0.0785
Epoch 58/100
50653/50653 [==============================] - 74s 1ms/step - loss: 3.5038 - accuracy: 0.1075 - val_loss: 3.6562 - val_accuracy: 0.0786
Epoch 59/100
50653/50653 [==============================] - 74s 1ms/step - loss: 3.5026 - accuracy: 0.1074 - val_loss: 3.6568 - val_accuracy: 0.0790
Epoch 60/100
50653/50653 [==============================] - 74s 1ms/step - loss: 3.5019 - accuracy: 0.1094 - val_loss: 

In [5]:
test_predict_sift = model.predict_classes(scaled_test_sift)
val_predict_sift = model.predict_classes(scaled_val_sift)

In [6]:
Y_test_sift = [int(i) for i in Y_test_sift]
Y_val_sift = [int(i) for i in Y_val_sift]

In [7]:
accuracy_score(test_predict_sift, Y_test_sift)#準確率

0.024498886414253896

In [8]:
accuracy_score(val_predict_sift, Y_val_sift)#精確度

0.017777777777777778